In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

In [3]:
# Carregar dados
data = pd.read_csv('data_/TSLA.csv')
data['Date'] = pd.to_datetime(data['Date'])

In [4]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-09,22.366667,22.900000,22.098000,22.568666,22.568666,81493500
1,2019-01-10,22.293333,23.025999,22.119333,22.997999,22.997999,90846000
2,2019-01-11,22.806000,23.227333,22.584667,23.150667,23.150667,75586500
3,2019-01-14,22.825333,22.833332,22.266666,22.293333,22.293333,78709500
4,2019-01-15,22.333332,23.253332,22.299999,22.962000,22.962000,90849000


In [5]:
data['Moving_Average'] = data['Close'].rolling(window=20).mean()

data['Distance_Percentage'] = ((data['Close'] - data['Moving_Average']) / data['Moving_Average']) * 100
data['Distance_Absolute'] = data['Close'] - data['Moving_Average']

print(data.head(25))

         Date       Open       High        Low      Close  Adj Close  \
0  2019-01-09  22.366667  22.900000  22.098000  22.568666  22.568666   
1  2019-01-10  22.293333  23.025999  22.119333  22.997999  22.997999   
2  2019-01-11  22.806000  23.227333  22.584667  23.150667  23.150667   
3  2019-01-14  22.825333  22.833332  22.266666  22.293333  22.293333   
4  2019-01-15  22.333332  23.253332  22.299999  22.962000  22.962000   
5  2019-01-16  22.985332  23.466667  22.900000  23.070000  23.070000   
6  2019-01-17  23.080667  23.433332  22.943333  23.153999  23.153999   
7  2019-01-18  21.533333  21.808666  19.982000  20.150667  20.150667   
8  2019-01-22  20.321333  20.533333  19.700001  19.927999  19.927999   
9  2019-01-23  19.500000  19.633333  18.779333  19.172667  19.172667   
10 2019-01-24  18.868668  19.578667  18.618668  19.434000  19.434000   
11 2019-01-25  19.625999  19.901333  19.303333  19.802668  19.802668   
12 2019-01-28  19.527332  19.830667  19.183332  19.758667  19.75

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [7]:
# Calculando média móvel e distância do fechamento à média
data['Moving_Average'] = data['Close'].rolling(window=20).mean()
data['Distance_From_MA'] = data['Close'] - data['Moving_Average']

# Definindo a coluna de destino: 1 para aumento no próximo dia e 0 para queda ou sem mudança
data['Target'] = (data['Close'].shift(-1) > data['Close']).astype(int)

# Removendo linhas NaN
data = data.dropna()

In [8]:
# Features e target
X = data[['Close', 'Distance_From_MA']]
y = data['Target']

# Divisão treino-teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [9]:
# Treinando um modelo RandomForest
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

RandomForestClassifier()

In [11]:
# Predições e avaliação
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.5160


In [12]:
# Criando sinais de trading
data['Signal'] = 0
data.loc[data.index[-len(y_pred):], 'Signal'] = y_pred
data['Position'] = data['Signal'].diff()

print(data.tail(50))

           Date        Open        High         Low       Close   Adj Close  \
1062 2023-03-29  193.130005  195.289993  189.440002  193.880005  193.880005   
1063 2023-03-30  195.580002  197.330002  194.419998  195.279999  195.279999   
1064 2023-03-31  197.529999  207.789993  197.199997  207.460007  207.460007   
1065 2023-04-03  199.910004  202.690002  192.199997  194.770004  194.770004   
1066 2023-04-04  197.320007  198.740005  190.320007  192.580002  192.580002   
1067 2023-04-05  190.520004  190.679993  183.759995  185.520004  185.520004   
1068 2023-04-06  183.080002  186.389999  179.740005  185.059998  185.059998   
1069 2023-04-10  179.940002  185.100006  176.110001  184.509995  184.509995   
1070 2023-04-11  186.690002  189.190002  185.649994  186.789993  186.789993   
1071 2023-04-12  190.740005  191.580002  180.309998  180.539993  180.539993   
1072 2023-04-13  182.960007  186.500000  180.940002  185.899994  185.899994   
1073 2023-04-14  183.949997  186.279999  182.009995 